<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/8_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.

1. 對 col1 和 col2 分群。
2. 計算每個群體中 col4 的相對誤差比率 v。
3. 提取每個群體中指定的 col3 對應的 v 值。
4. 將每個群體中提取出的數據拼成一個新的 DataFrame。

In [ ]:
import pandas as pd
import numpy as np

# 示例數據
data = {
    'col1': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C'],
    'col2': ['X', 'X', 'Y', 'Y', 'Y', 'Z', 'Z', 'Z', 'Z'],
    'col3': [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'col4': [10, 12, 14, 10, 11, 13, 9, 10, 15]
}
df = pd.DataFrame(data)

# 指定要提取的 col3 值
specified_col3_values = [1, 3]

# 用於存儲結果的列表
result_list = []

# 對 col1 和 col2 分群，並在循環內計算相對誤差比率和提取值
for (name, group) in df.groupby(['col1', 'col2']):
    # 計算中位數
    median_col4 = group['col4'].median()
    # 計算相對誤差比率
    group['v'] = np.abs(group['col4'] - median_col4) / median_col4
    # 提取指定的 col3 值對應的 v 值
    filtered_group = group[group['col3'].isin(specified_col3_values)]
    # 將結果 append 到列表中
    result_list.append(filtered_group)

# 將結果拼接成一個新的 DataFrame
result_df = pd.concat(result_list, ignore_index=True)

# 打印結果
print(result_df)


### 1-1. **在最終結果中添加三個新的列**



  - rank1：根據原始分群計算的順序，依次排序給出序號。
  - rank2：根據每個群體中 v 的平均值排序，從大到小。
  - rank3：根據每個群體中 v 的最大值排序，從大到小。

In [ ]:
import pandas as pd
import numpy as np

# 示例數據
data = {
    'col1': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C'],
    'col2': ['X', 'X', 'Y', 'Y', 'Y', 'Z', 'Z', 'Z', 'Z'],
    'col3': [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'col4': [10, 12, 14, 10, 11, 13, 9, 10, 15]
}
df = pd.DataFrame(data)

# 指定要提取的 col3 值
specified_col3_values = [1, 3]

# 用於存儲結果的列表
result_list = []

# 用於存儲每個群體的統計信息
group_stats = []

# 對 col1 和 col2 分群，並在循環內計算相對誤差比率和提取值
for idx, (name, group) in enumerate(df.groupby(['col1', 'col2']), start=1):
    # 計算中位數
    median_col4 = group['col4'].median()
    # 計算相對誤差比率
    group['v'] = np.abs(group['col4'] - median_col4) / median_col4
    # 提取指定的 col3 值對應的 v 值
    filtered_group = group[group['col3'].isin(specified_col3_values)]
    # 添加 rank1
    filtered_group['rank1'] = idx
    # 添加群體的中位數
    filtered_group['median'] = median_col4
    # 計算群體的 v 的統計信息
    mean_v = group['v'].mean()
    max_v = group['v'].max()
    group_stats.append((name, mean_v, max_v))
    # 將結果 append 到列表中
    result_list.append(filtered_group)

# 將結果拼接成一個新的 DataFrame
result_df = pd.concat(result_list, ignore_index=True)

# 將群體的統計信息轉換成 DataFrame
stats_df = pd.DataFrame(group_stats, columns=['group', 'mean_v', 'max_v'])

# 計算 rank2 和 rank3
stats_df['rank2'] = stats_df['mean_v'].rank(method='dense', ascending=False).astype(int)
stats_df['rank3'] = stats_df['max_v'].rank(method='dense', ascending=False).astype(int)

# 合併 rank2 和 rank3 到結果 DataFrame
result_df = result_df.merge(stats_df[['group', 'rank2', 'rank3']], left_on='rank1', right_index=True, how='left')

# 打印結果
print(result_df)


### 1-2. 再計算偏差的時候，會出現一種狀況，就是**col4有些值是''**，那麼我想要忽略這些去做，該如何做

Ans: 計算之前將這些值過濾掉。

具體步驟：
1. 將空字符串轉換為 NaN：使用 pd.to_numeric 方法將 col4 中的空字符串轉換為 NaN。
2. 對 col1 和 col2 分群並計算 v 值。
3. 在每個群體內計算 v 的平均值和最大值。
4. 提取每個群體中指定的 col3 值的有效數據並計算 v 值。
5. 將結果拼接成一個新的 DataFrame。

In [ ]:
import pandas as pd
import numpy as np

# 示例數據，包含空字符串值
data = {
    'col1': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C'],
    'col2': ['X', 'X', 'Y', 'Y', 'Y', 'Z', 'Z', 'Z', 'Z'],
    'col3': [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'col4': [10, 12, '', 10, 11, 13, 9, '', 15]
}
df = pd.DataFrame(data)

# 將空字符串轉換為NaN
df['col4'] = pd.to_numeric(df['col4'], errors='coerce')

# 指定要提取的 col3 值
specified_col3_values = [1, 3]

# 用於存儲結果的列表
result_list = []

# 用於存儲每個群體的統計信息
group_stats = []

# 對 col1 和 col2 分群，並在循環內計算相對誤差比率和提取值
for idx, (name, group) in enumerate(df.groupby(['col1', 'col2']), start=1):
    # 去掉 col4 中的 NaN 值
    valid_group = group.dropna(subset=['col4'])

    # 計算中位數
    median_col4 = valid_group['col4'].median()

    # 計算相對誤差比率
    valid_group['v'] = np.abs(valid_group['col4'] - median_col4) / median_col4

    # 提取指定的 col3 值對應的 v 值，且 col4 必須不是 NaN
    filtered_group = valid_group[valid_group['col3'].isin(specified_col3_values)]

    # 添加 rank1
    filtered_group['rank1'] = idx
    # 添加群體的中位數
    filtered_group['median'] = median_col4

    # 計算群體的 v 的統計信息
    mean_v = valid_group['v'].mean()
    max_v = valid_group['v'].max()
    group_stats.append((name, mean_v, max_v))

    # 將結果 append 到列表中
    result_list.append(filtered_group)

# 將結果拼接成一個新的 DataFrame
result_df = pd.concat(result_list, ignore_index=True)

# 將群體的統計信息轉換成 DataFrame
stats_df = pd.DataFrame(group_stats, columns=['group', 'mean_v', 'max_v'])

# 計算 rank2 和 rank3
stats_df['rank2'] = stats_df['mean_v'].rank(method='dense', ascending=False).astype(int)
stats_df['rank3'] = stats_df['max_v'].rank(method='dense', ascending=False).astype(int)

# 合併 rank2 和 rank3 到結果 DataFrame
result_df = result_df.merge(stats_df[['group', 'rank2', 'rank3']], left_on='rank1', right_index=True, how='left')

# 打印結果
print(result_df)


In [ ]:
    col1  col2    col3  col4    v     rank1    rank2   rank3
0    A    X     1  10.0  0.166667      1      2      2
1    A    X     3  12.0  0.000000      1      2      2
2    B    Y     1  10.0  0.095238      2      3      3
3    B    Y     3  13.0  0.142857      2      3      3
4    C    Z     1   9.0  0.333333      3      1      1
5    C    Z     3  15.0  0.666667      3      1      1


* **對result_df根據rank1 提取相對應的col1,col2值**

In [ ]:
result_df = pd.DataFrame(data)

# 提取相對應的col1, col2值
unique_rank1 = result_df['rank1'].unique()

for rank in unique_rank1:
    subset = result_df[result_df['rank1'] == rank]
    col1_value = subset['col1'].iloc[0]
    col2_value = subset['col2'].iloc[0]
    print(f"Rank1: {rank}, col1: {col1_value}, col2: {col2_value}")